<a href="https://colab.research.google.com/github/CermakM/OpenNMT-tf/blob/master/jabber-wocky-opennmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
# import opennmt as onmt

In [1]:
!git clone https://github.com/CermakM/OpenNMT-tf

Cloning into 'OpenNMT-tf'...
remote: Enumerating objects: 245, done.
remote: Counting objects: 100% (245/245), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 7453 (delta 167), reused 106 (delta 56), pack-reused 7208
Receiving objects: 100% (7453/7453), 11.02 MiB | 13.21 MiB/s, done.
Resolving deltas: 100% (5652/5652), done.


In [3]:
cd OpenNMT-tf

/content/OpenNMT-tf


In [0]:
tf.enable_eager_execution()

In [0]:
"""Main script."""

import argparse
import json
import os
import six

import tensorflow as tf

from google.protobuf import text_format

from opennmt import __version__
from opennmt.models import catalog
from opennmt.runner import Runner
from opennmt.config import load_model, load_config
from opennmt.utils.misc import classes_in_module

tf.logging.set_verbosity('DEBUG')

# Load and merge run configurations.
config = load_config(['reference-config.yaml'])

In [9]:
is_chief = True

if is_chief and not tf.gfile.Exists(config["model_dir"]):
  tf.logging.info("Creating model directory %s", config["model_dir"])
  tf.gfile.MakeDirs(config["model_dir"])

model = load_model(
    config["model_dir"],
    model_name='Transformer',
    serialize_model=is_chief)
session_config = tf.ConfigProto(
    intra_op_parallelism_threads=0,
    inter_op_parallelism_threads=0,
    gpu_options=tf.GPUOptions(
        allow_growth=False))

INFO:tensorflow:Creating model directory jabber-wocky/


In [0]:
args = type('args', (), {})
args.run = 'train'
args.predictions_file = ""
args.checkpoint_path  = None

runner = Runner(
    model,
    config,
    seed=42,
    num_devices=1,
    session_config=session_config,
    auto_config=False)

if args.run == "train_and_eval":
  runner.train_and_evaluate(checkpoint_path=args.checkpoint_path)
elif args.run == "train":
  runner.train(checkpoint_path=args.checkpoint_path)
elif args.run == "eval":
  runner.evaluate(checkpoint_path=args.checkpoint_path)
elif args.run == "infer":
  if not args.features_file:
    parser.error("--features_file is required for inference.")
  elif len(args.features_file) == 1:
    args.features_file = args.features_file[0]
  runner.infer(
      args.features_file,
      predictions_file=args.predictions_file,
      checkpoint_path=args.checkpoint_path,
      log_time=args.log_prediction_time)
elif args.run == "export":
  runner.export(
      checkpoint_path=args.checkpoint_path,
      export_dir_base=args.export_dir_base)
elif args.run == "score":
  if not args.features_file:
    parser.error("--features_file is required for scoring.")
  if not args.predictions_file:
    parser.error("--predictions_file is required for scoring.")
  runner.score(
      args.features_file,
      args.predictions_file,
      checkpoint_path=args.checkpoint_path)

INFO:tensorflow:Using parameters: {
  "data": {
    "eval_features_file": "data/src-val.txt",
    "eval_labels_file": "data/tgt-val.txt",
    "source_words_vocabulary": "data/src-vocab.txt",
    "target_words_vocabulary": "data/tgt-vocab.txt",
    "train_features_file": "data/src-train.txt",
    "train_labels_file": "data/tgt-train.txt"
  },
  "eval": {
    "batch_size": 30,
    "eval_delay": 7200,
    "exporters": "last",
    "external_evaluators": "BLEU",
    "num_threads": 1,
    "prefetch_buffer_size": 1,
    "save_eval_predictions": false
  },
  "infer": {
    "batch_size": 10,
    "n_best": 1,
    "num_threads": 1,
    "prefetch_buffer_size": 1,
    "with_alignments": null,
    "with_scores": false
  },
  "model_dir": "jabber-wocky/",
  "params": {
    "average_loss_in_time": false,
    "beam_width": 5,
    "clip_gradients": 5.0,
    "decay_rate": 0.7,
    "decay_step_duration": 1,
    "decay_steps": 10000,
    "decay_type": "exponential_decay",
    "guided_alignment_type": null,